In [1]:
from lightning_gans.models.resnet import conv_downsample
from lightning_gans.models.discriminators import Discriminator
import timm
import pytorch_lightning as pl
import torch
import torch.nn as nn
import albumentations as A
from albumentations.pytorch import ToTensorV2
from lightning_gans.data.dataloader import MonetDataset, AbstractArtDataset
from pytorch_lightning.loggers import TensorBoardLogger
import matplotlib.pyplot as plt
import numpy as np
from pytorch_lightning.callbacks import ModelCheckpoint
import tqdm

from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seed for reproducibility
#manualSeed = 999
#manualSeed = random.randint(1,if norm:
        layers.append(norms[norm](num_features=out_c, affine=True))
    if dropout:
        layers.append(torch.nn.Dropout2d(p=0.5, inplace=True))
    if activation:
        layers.append(activations[activation])
    return torch.nn.Sequential(*layers) 10000) # use if you want new results
#print("Random Seed: ", manualSeed)
#random.seed(manualSeed)
#torch.manual_seed(manualSeed)

In [2]:
activations = {
    "leaky_relu": torch.nn.LeakyReLU(0.2,inplace=True),
    "relu": torch.nn.ReLU(inplace=True),
    "tanh": torch.nn.Tanh()
}
norms = {"batch": torch.nn.BatchNorm2d, "instance": torch.nn.InstanceNorm2d}


def weights_init(m):
    """Initializes weights with a 0 mean and 0.02 stdev

    Args:
        m (torch.nn.Module): torch module whose weights will be initialized
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    if classname.find('InstanceNorm') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    if classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

        
def conv_upsample(ks, in_c, out_c, st=2, activation="leaky_relu", norm="batch", dropout=False):
    padding = {
        2:[0,0],
        3:[1,1],
        4:[1,0],
        5:[2,1]
    }
    if st==1:
        padding[4][0]=0
    layers = [
        torch.nn.ConvTranspose2d(in_c, out_c, ks, st, padding=padding[ks][0], output_padding=padding[ks][1],
                                 bias=False),
        #torch.nn.UpsamplingBilinear2d(scale_factor=2),
        #torch.nn.Conv2d(in_c, out_c, kernel_size=1, stride=1, bias=False)
    ]
    if norm:
        layers.append(norms[norm](num_features=out_c, affine=True))
    if dropout:
        layers.append(torch.nn.Dropout2d(p=0.5, inplace=True))
    if activation:
        layers.append(activations[activation])
    return torch.nn.Sequential(*layers)

In [13]:
class GAN(pl.LightningModule):
    def __init__(self,latent_size):
        super().__init__()
        self.latent_size = latent_size
        
        self.generator = None
        self.discriminator = None
        self._create_model()
        self.generator.apply(weights_init)
        self.discriminator.apply(weights_init)
        
        self.generator_optimizer=None
        self.discriminator_optimizer=None
        self.automatic_optimization=False
        
        self.loss = torch.nn.BCELoss()
        self.fixed_noise = torch.randn(64, self.latent_size, 1, 1)
        
    def _create_model(self):
        self.generator = torch.nn.Sequential(
            # input = Bx500x1x1
            conv_upsample(4,in_c=self.latent_size,out_c=1024,st=1,dropout=True),
            # shape = Bx1024x4x4
            conv_upsample(4,in_c=1024,out_c=512,dropout=True),
            # shape = Bx512x8x8
            conv_upsample(4,in_c=512,out_c=256,dropout=True),
            # shape = Bx256x16x16
            conv_upsample(4,in_c=256,out_c=128,dropout=True),
            # shape = Bx128x32x32
            conv_upsample(4,in_c=128,out_c=3,activation="tanh",norm=None),
            # shape = Bx3x64x64
        )
        """
        self.discriminator = torch.nn.Sequential(
            timm.create_model("resnet18",num_classes=1,pretrained=True),
            torch.nn.Sigmoid()
        )"""
        nc = 3
        ndf = 64
        self.discriminator = torch.nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
        
    def configure_optimizers(self):
        self.generator_optimizer = torch.optim.Adam(self.generator.parameters(),
                                                        lr=2e-4,
                                                        betas=(0.5, 0.999))
        self.discriminator_optimizer = torch.optim.Adam(self.discriminator.parameters(),
                                                        lr=2e-4,
                                                        betas=(0.5, 0.999))
        return [self.generator_optimizer, self.discriminator_optimizer]
    
    def training_step(self, train_batch, batch_idx):
        g_opt, d_opt = self.optimizers()
        real_images,_ = train_batch
        batch_size = real_images.shape[0]
        
        r1, r2 = 0.75, 0.83
        real_label = (r1 - r2) * torch.randn(1) + r2
        real_label = real_label.item()
        fake_label = 0.1
        
        targets = torch.full((batch_size,),fill_value=real_label,dtype=torch.float,device=self.device)
        
        ####### 1 - UPDATE DISCRIMINATOR #######
        d_opt.zero_grad()
        self.discriminator.zero_grad()
        # REAL IMAGE BATCH PASSED THROUGH D
        preds = self.discriminator(real_images)
        preds = preds.squeeze()
        D_x = preds.mean().item()
        
        real_loss = self.loss(preds,targets)
        self.manual_backward(real_loss)
        # FAKE IMAGE BATCH PASSED THROUGH D
        generator_inputs = torch.randn(batch_size,self.latent_size,1,1,device=self.device)
        fake_images = self.generator(generator_inputs)
        # PREDICTIONS ON  FAKE IMAGES
        preds = self.discriminator(fake_images.detach())
        preds = preds.squeeze()
        D_G_z1 = preds.mean().item()
        
        targets.fill_(fake_label)
        fake_loss = self.loss(preds,targets)
        self.manual_backward(fake_loss)
        d_opt.step()
        loss_d = real_loss+fake_loss
        #self.manual_backward(loss_d)
        #d_opt.step()
        
        ####### 2 - UPDATE GENERATOR #######
        g_opt.zero_grad()
        self.generator.zero_grad()
        targets.fill_(real_label)
        # PREDICTIONS ON FAKE IMAGES
        preds = self.discriminator(fake_images)
        preds = preds.squeeze()
        D_G_z2 = preds.mean().item()
        generator_loss = self.loss(preds,targets)
        self.manual_backward(generator_loss)
        g_opt.step()
        
        self.log_dict({
            "loss_d":loss_d.item(),
            "loss_g":generator_loss.item(),
            "D_x":D_x,
            "D_G_z1":D_G_z1,
            "D_G_z2":D_G_z2,
        }, prog_bar=True)
        
    def validation_step(self, batch, batch_idx):
        if batch_idx==0:
            fake = self.generator(self.fixed_noise.to(self.device)).detach().cpu()
            grid = vutils.make_grid(fake, padding=2, normalize=True)
            self.logger.experiment.add_image('fixed_noise_outputs', grid, self.global_step)

In [15]:
batch_size = 128
workers =16
version = "64x64Images_NoNormGenLast_1"
name="DCGAN_Abstract"
logger = TensorBoardLogger(
            "../data/interim/tboard/",
            name=name,
            version=version,
        )
epochs = 1000
checkpoint_callback = ModelCheckpoint(
            save_top_k=3,
            verbose=True,
            monitor="loss_g",
            mode="min",
            filename="model-epoch{epoch:02d}-val_loss_g_{loss_g:.2f}",
            auto_insert_metric_name=False,
            dirpath="../data/interim/tboard/{}/{}/models/".format(name,version),
        )


transforms = A.Compose([
            A.Resize(64, 64),
            A.Normalize(mean=0, std=1, max_pixel_value=255),
            ToTensorV2(),
        ])
#train_dataset = MonetDataset(dataroot="../data/raw/monet/",transforms=transforms,shuffle=True,complete_dataset=False)

train_dataset = AbstractArtDataset(dataroot="../data/raw/art/",
                             transforms=transforms,
                             size=(64,64))

In [16]:
# Create the dataloader
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               num_workers=workers)
# Create the generator
model = GAN(100)
trainer = pl.Trainer(
    logger=logger,
    max_epochs=epochs,
    log_every_n_steps=1,
    gpus=1,
    callbacks=[checkpoint_callback],
    enable_checkpointing=True,
    enable_progress_bar=True,
    val_check_interval=10,
    #resume_from_checkpoint="../data/interim/tboard/DCGAN/64x64Images_NoNormGenLast/models/model-epoch81-val_loss_g_1.06.ckpt"
)
trainer.fit(model, train_dataloader=train_dataloader, val_dataloaders=train_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/aahan/anaconda3/envs/pytorch/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:735: LightningDeprecationWarning: `trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.fit(train_dataloaders)` instead. HINT: added 's'
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type       | Params
---------------------------------------------
0 | generator     | Sequential | 12.7 M
1 | discriminator | Sequential | 2.8 M 
2 | loss          | BCELoss    | 0     
---------------------------------------------
15.4 M    Trainable params
0         Non-trainable params
15.4 M    Total params
61.696    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 0, global step 9: loss_g reached 2.43019 (best 2.43019), saving model to "/home/aahan/git_repos/LightningGan/data/interim/tboard/DCGAN_Abstract/64x64Images_NoNormGenLast_1/models/model-epoch00-val_loss_g_2.43.ckpt" as top 3


...

Epoch 104, global step 2401: loss_g was not in top 3


Validating: 0it [00:00, ?it/s]

Epoch 104, global step 2411: loss_g was not in top 3


Validating: 0it [00:00, ?it/s]

/home/aahan/anaconda3/envs/pytorch/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
m = GAN.load_from_checkpoint("../data/interim/tboard/DCGAN_Monet/64x64Images_NoNormGenLast_4/models/model-epoch140-val_loss_g_1.75.ckpt",latent_size=100)

In [ ]:
tmp = torch.randn(1,100,1,1,requires_grad=False,device=model.device)
o = m.generator(tmp.detach())
img = o * 127.5 + 127.5
img = img.squeeze().permute(1, 2, 0).cpu().detach().numpy().astype(np.uint8)
plt.imshow(img)
plt.show()

In [ ]:
# Root directory for dataset
dataroot = "../data/raw/monet/"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 1000

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5# Create the dataloader

transforms = A.Compose([
            A.Resize(64, 64),
            A.Normalize(mean=0, std=1, max_pixel_value=255),
            ToTensorV2(),
        ])
train_dataset = MonetDataset(dataroot="../data/raw/monet/",
                         transforms=transforms,complete_dataset=False)


dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               num_workers=workers)

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [ ]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [ ]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.02.
netG.apply(weights_init)

# Print the model
print(netG)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)

In [ ]:
# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 50 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()